<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notebook-setup" data-toc-modified-id="Notebook-setup-1">Notebook setup</a></span></li><li><span><a href="#Python-libraries-for-geographic-data" data-toc-modified-id="Python-libraries-for-geographic-data-2">Python libraries for geographic data</a></span></li><li><span><a href="#Soil-uranium-parts-per-million" data-toc-modified-id="Soil-uranium-parts-per-million-3">Soil uranium parts per million</a></span><ul class="toc-item"><li><span><a href="#Add-column-'Uppm'-to-GeoPandasDataFrame" data-toc-modified-id="Add-column-'Uppm'-to-GeoPandasDataFrame-3.1">Add column 'Uppm' to GeoPandasDataFrame</a></span></li></ul></li><li><span><a href="#From-areal-maps-to-graphs" data-toc-modified-id="From-areal-maps-to-graphs-4">From areal maps to graphs</a></span></li><li><span><a href="#Representing-the-neighborhood-network-as-an-adjacency-list" data-toc-modified-id="Representing-the-neighborhood-network-as-an-adjacency-list-5">Representing the neighborhood network as an adjacency list</a></span></li></ul></div>

### Notebook setup

In [ ]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
# gis data
import geopandas as gpd
import libpysal as sa
# plotting libs
import matplotlib.pyplot as plt
import splot as splt
import plotnine as p9
%matplotlib inline

In [ ]:
# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')
# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=14),
        axis_title_x=p9.element_text(size=12),
        axis_title_y=p9.element_text(size=12),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8)
       )
)
xlabels_90 = p9.theme(axis_text_x = p9.element_text(angle=90, hjust=1))

In [ ]:
# keep notebook outputs clean - demos only
import logging
logging.getLogger('cmdstanpy').setLevel(logging.CRITICAL)

### Python libraries for geographic data

Geographic information systems (GIS) data is any item which has a geographic location, either a single point or a set of bounding polygons.  In order to manage, analyze, and visualize GIS data, we use specialized packages which can do the geographic math.  In this notebook we use the following packages:

- GeoPandas - manages a set of GIS records in tabular format
- libpysal - spatial analysis package which can analyze distance between locations
- splot - plots for libpysal objects
- plotnine - object `geom_map`

Cartographic data (maps) are encoded as a set of records, one per map region.  The [shapefile format](https://en.wikipedia.org/wiki/Shapefile) is an open specification used to insure interoperatility among GIS software packages.  When items in a dataset contain location labels, it is necessary to obtain a set of shapefiles for the corresponding map.

The shapefiles for US counties are available from the [US Census Bureau](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html).
For this analysis, we are using shapefiles where the boundary information is specified with the lowest possible resolution; this greatly speeds up analysis and plotting.
These can be downloaded via URL: https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_20m.zip

In [ ]:
shpfile = os.path.join('geo_data','cb_2018_us_county_20m', 'cb_2018_us_county_20m.shp')
us_geodata = gpd.read_file(shpfile)
print(type(us_geodata))
us_geodata.head(3)

The GeoDataFrame object can be manipulated like a Pandas DataFrame object.

For our analysis, we are only using data from the continental US.  We drop the records for Alaska, Hawaii, Puerto Rico, and the US Virgin Islands.

In [ ]:
# no data on ALASKA ('02'), HAWAII ('15'), PR ('72'), USVI ('78')
islands = ['02', '15', '72', '78']
us_geodata = us_geodata[~us_geodata.STATEFP.isin(islands)]

To see the result, we use plotnine's `geom_map` object.

In [ ]:
(p9.ggplot()
 + p9.geom_map(data=us_geodata, fill='lightblue')
 + p9.theme(figure_size=(16, 9))
)

### Soil uranium parts per million

County-level measurements of soil uranium in parts per million is used as a predictor in the Gelman and Hill radon analysis.   This dataset comes from the book's website of data and codes.

In [ ]:
us_uranium = pd.read_csv(os.path.join('data','raw_uranium.csv'),
                        usecols=['stfips', 'ctfips', 'st', 'cty', 'Uppm'],
                        skipinitialspace=True,
                        ).drop_duplicates().convert_dtypes()
us_uranium.head(3)

In [ ]:
print(f'median soil uranium level: {us_uranium.Uppm.median()}')
print(f'average soil uranium level: {us_uranium.Uppm.mean()}')
p9.ggplot(data=us_uranium, mapping=p9.aes(x='Uppm')) + p9.stats.stat_density(geom='line')

#### Add column 'Uppm' to GeoPandasDataFrame

To visualize the soil levels of uranium (in parts per million), we merge the EPA soil uranium measurements into the GeoPandasDataFrame. 

In order to do a database-style join on the two tables, we need to first add a common key to both tables.
[FIPS code](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) are numbers which uniquely identify geographic areas. The GIS dataset combines the state and county FIPS codes into a national-level `GEOID`.
We need to do the same for the uranium data.

Once we have a common key, we add the county-level soil uranium levels to the GIS data via the [DataFrame.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method.

In [ ]:
# create merge column
us_uranium['FIPS'] = us_uranium.stfips*1000 + us_uranium.ctfips

# GEOID should be numeric
us_geodata = us_geodata.astype({'GEOID': 'int32'}, copy=False)

# left join because Uranium dataset is missing one record
us_geodata = us_geodata.merge(us_uranium, how='left', left_on='GEOID', right_on='FIPS')

# cleanup missing value so that we can use column as fill value
us_geodata = us_geodata.astype({'Uppm': 'float64'})
us_geodata.Uppm.fillna(value=0, inplace=True)

# drop columns for cleaner display
us_geodata.drop(columns=['STATEFP','COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'LSAD', 'ALAND', 'AWATER',
                       'stfips', 'ctfips', 'cty', 'FIPS'], inplace=True)
us_geodata.rename(columns={'st':'STATE','NAME': 'county'}, inplace=True)

In [ ]:
us_geodata.head(3)

**Add log_uranium**

Soil uranium levels are always positive, i.e., bounded by zero.  By working with the log uranium level, we have an variable whose range is unbounded.  This provides modeling flexibility, and improves Stan performance.

For inference, we will use log_uranium.   Log of 0 is -inf, so we set 0 values to 0.1.

In [ ]:
us_geodata['Uppm'] = us_geodata['Uppm'].where(us_geodata['Uppm'] > 0.1, other=0.1)
us_geodata['log_uranium'] = np.log(us_geodata['Uppm'])

Now we plot our map, this time using the soil uranium levels as the fill color.
The darkest purple colored areas have recorded soil uranium level of zero.
This includes all of Connecticut.

In [ ]:
(p9.ggplot()
 + p9.geom_map(data=us_geodata, mapping=p9.aes(fill='log_uranium'))
 + p9.scale_fill_cmap(cmap_name='plasma')
 + p9.theme(figure_size=(16, 9))
)

**Density plots, uranium levels US vs MN**

Overall, the soil uranium levels are below the median soil uranium level in the US.  We overlay density plots of MN in green and the entire US in purple.

In [ ]:
mn_geodata = us_geodata[us_geodata['STATE']=='MN']

(p9.ggplot() 
 + p9.stats.stat_density(data=us_geodata, mapping=p9.aes(x='log_uranium'), geom='line', color='purple')
 + p9.stats.stat_density(data=mn_geodata, mapping=p9.aes(x='log_uranium'), geom='line', color='green')
)

**Geographic variation**

If we plot just the counties in Minnesota, we see that there is a south to north and west to east decrease in soil uranium levels. 

In [ ]:
(p9.ggplot()
 + p9.geom_map(data=mn_geodata, mapping=p9.aes(fill='log_uranium'))
 + p9.theme(figure_size=(10, 8))
 + p9.scale_fill_cmap(cmap_name='plasma',
                      limits=[min(us_geodata.log_uranium),
                              max(us_geodata.log_uranium)])  # same scale as entire US plot
)

### From areal maps to graphs

By defining a neighbor relationship between area regions, we convert a map to a graph.

**Computing the neighborhood network with libpysal**

We need to compute the adjacency network between the regions in our map.
We do this with [libpysal](https://pysal.org/libpysal/), a Python library for spatial analysis, which we have imported as `sa`.
To visualize this graph overlaid on the county map, we use [splot](https://splot.readthedocs.io/en/latest/), a lightweight visualization interface.

There are several ways to define the neighbor relationship, see https://pysal.org/libpysal/api.html#spatial-weights.
Here we use the *Rook* metric:  regions which share a common line boundary are neighbors.  This returns a spatial weights object, [weights.W](https://pysal.org/libpysal/generated/libpysal.weights.W.html#libpysal-weights-w)

In [ ]:
# neighbors are counties which have a common line boundary
nb_mn = sa.weights.Rook(mn_geodata['geometry'])

In [ ]:
from splot.libpysal import plot_spatial_weights
plot_spatial_weights(nb_mn, mn_geodata)

In [ ]:
print(f'nodes: {nb_mn.n}')
print(f'number of components: {nb_mn.n_components}')
print(f'islands? {nb_mn.islands}')
print(f'max nmber of neighbors per node: {nb_mn.max_neighbors}')
print(f'mean nmber of neighbors per node: {nb_mn.mean_neighbors}')

In [ ]:
# plot histogram of number of neighbors per county
(p9.ggplot()
 + p9.geom_histogram(mapping=p9.aes(x=list(nb_mn.cardinalities.values())),
                     binwidth=1, fill='lightblue', color='blue')
 + p9.scales.scale_x_continuous(limits=(0,11), minor_breaks=None)
 + p9.xlab('Minnesota number of neighbors per county')
 + p9.ylab('')
)

We can do the same visualization for the entire continental US.

In [ ]:
nb_us = sa.weights.Rook(us_geodata['geometry'])
splt.libpysal.plot_spatial_weights(nb_us, us_geodata)

In [ ]:
print(f'nodes: {nb_us.n}')
print(f'number of components: {nb_us.n_components}')
print(f'islands? {nb_us.islands}')
print(f'max nmber of neighbors per node: {nb_us.max_neighbors}')
print(f'mean nmber of neighbors per node: {nb_us.mean_neighbors}')

In [ ]:
# plot histogram of number of neighbors per county
(p9.ggplot()
 + p9.geom_histogram(
     mapping=p9.aes(x=list(nb_us.cardinalities.values())),
     binwidth= 1, fill='lightblue', color='blue')
 + p9.xlab('USA number of neighbors per county')
 + p9.ylab('')
)

### Representing the neighborhood network as an adjacency list

We use the libpysal Weights object method [to_adjlist](https://pysal.org/libpysal/generated/libpysal.weights.W.html#libpysal.weights.W.to_adjlist) to extract neighbor information in the form of a list of adjacent nodes.
All the indices are 0-based and match the order in which they occur in the GeoPandasDataFrame object `mn_geodata`, which is sorted by county name.

In [ ]:
nm_adj =  nb_mn.to_adjlist(remove_symmetric=True)
print(f'County name index 0? {mn_geodata.county.values[0]}')
nm_adj.head(3)

To check our work, we first add labels to the map of Minnesota, then check that the adjacency graph corresponds to the map.

We use plotnine's `geom_text` to add text labels to the center of each county.
This requires computing the center of each county.
The GeoDataFrame.geometry.centroid property has the center coordinates of polygons.

In [ ]:
county_pts = mn_geodata.geometry.centroid.to_crs(mn_geodata.crs)

(p9.ggplot()
 + p9.geom_map(data=mn_geodata, mapping=p9.aes(fill='log_uranium'))
 + p9.geom_text(mapping=p9.aes(x=county_pts.x, y=county_pts.y, label=list(mn_geodata.county)),
                size=6, ha='center')
 + p9.scale_fill_cmap(cmap_name='plasma',
                      limits=[min(us_geodata.log_uranium),
                              max(us_geodata.log_uranium)])
 + p9.theme(figure_size=(10, 9))
)

We expect 'Cook' county to have 1 neighbor, 'Lake' county.

In [ ]:
print(f'Cook county: {mn_geodata[mn_geodata.county == "Cook"].index[0]}, Lake county: {mn_geodata[mn_geodata.county == "Lake"].index[0]}')
nm_adj[nm_adj.focal == 15]